# Number Partitioning 2
Solve the number partitioning problem using DWave's QUBO generator

Given a set of numbers, partition the set into 2 subsets such that the 2 sums are as close as possible

In [1]:
import dimod

import pyqubo
from pyqubo import Spin, Binary, Constraint

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import LazyEmbeddingComposite

import numpy as np
import numpy.random as random

import time

## Function for solving QUBO
Given a hamiltonian $H$ in QUBO form, solve for lowest energy solution

In [20]:
def qdict_to_matrix(qubo,qubits):
    Q = np.zeros([len(qubits),len(qubits)])
    for (i,j) in qubo:
        x = int(i[1:i.find('.')])
        y = int(j[1:i.find('.')])
        Q[x,y] = qubo[(i,j)]
    return Q

def solve_qubo(H,
               qubits,
               sampler="CPU", # CPU or QPU
               k=10,
               chain_strength=None):
    """
    Given a hamiltonian, solves the quadratic unconstrained binary
    optimization (QUBO) problem given by
    
        minimize sum(x[i] * Q[i,j] * x[j]
                     for i in range(N),
                     for j in range(i+1, N))
    
    Uses compile() to convert the hamiltonian into matrix (dictionary) form.
    dimod.SimulatedAnnealingSampler is then used, which solves the problem k times through simulated
    annealing (on a regular CPU). This method returns the best solution found.
    """
    assert sampler in ["CPU", "QPU"]
    
    model = H.compile()
    qubo, offset = model.to_qubo()
    bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,offset=offset)
    
#     Q = qdict_to_matrix(qubo,qubits)
#     n = Q.shape[0]
#     nz = len(Q[Q!=0])
    n = len(qubits)
    nz = 900
    print("Solving QUBO problem (%d vars, %d nz) on %s..." % (n, nz, sampler))
    
    start = time.time()
    
    if sampler == "CPU":
        sampler = dimod.SimulatedAnnealingSampler()
        response = sampler.sample(bqm, num_reads=k)
    else:
        if chain_strength is None:
            chain_strength = int(10 * max(qubo.values()))
        sampler = LazyEmbeddingComposite(DWaveSampler(solver="Advantage_system6.1"))
        response = sampler.sample(bqm, num_reads=k, chain_strength=chain_strength)
        
    elapsed = time.time() - start
    
    print("Solved in %.2f seconds" % elapsed)
    solution = min(response.data(["sample", "energy"]), key=lambda s: s.energy)
    return solution, response

## Create set of integers

In [21]:
n = 100
random.seed(42)
S = random.randint(11,size=(n))
print(S)

[ 6  3 10  7  4  6  9  2  6 10 10  7  4  3  7  7  2  5  4  1  7  5  1  4
  0  9  5  8  0 10 10  9  2  6  3  8  2  4  2  6  4  8  6  1  3  8  1  9
  8  9  4  1  3  6  7  2  0  3  1  7  3  1  5  5  9  3  5  1  9  1  9  3
  7  6  8  7  4  1  4  7  9  8  8  0  8  6  8  7  0  7  7 10  2  0  7  2
  2  0 10  4]


## Define hamiltonian
We want to minimize the difference between the sum of each set

diff$^2=(c-2\sum_js_jx_j)^2$

Where $c$ is the sum of all the elements in the set $S$

In [22]:
# define spins/binary vars
spins = []
for i in range(len(S)):
    spins.append(Binary(f'x{i}.{S[i]}'))

# sum set S
c = np.sum(S)

# define hamiltonian
sumSx = 0
for i in range(len(spins)):
    sumSx += spins[i]*S[i]
    
H = (c-2*sumSx)**2

## Solve on CPU

In [23]:
k=10
# solve problem
solution1, response1 = solve_qubo(H,spins,"CPU",k)

# display result
print(response1)
# for smple, energy, num_occ in response.data(['sample','energy','num_occurrences']):
#     print(num_occ)

# display sums
print("")
S0 = [int(j[j.find('.')+1:]) for (j, xi) in solution1.sample.items() if xi >  0.5]
S1 = [int(j[j.index('.')+1:]) for (j, xi) in solution1.sample.items() if xi <= 0.5]

print('S:', S)
print('S0:',S0)
print('S1:',S1)

print("")

print("sum(S0) %8d" % sum(S0))
print("sum(S1) %8d" % sum(S1))

Solving QUBO problem (100 vars, 900 nz) on CPU...
Solved in 54.78 seconds
  x0.6 x1.3 x10.10 x11.7 x12.4 x13.3 x14.7 x15.7 x16.2 ... x99.4 energy num_oc.
0    1    1      1     1     0     0     0     0     0 ...     1    0.0       1
1    1    1      0     1     0     1     0     0     0 ...     0    0.0       1
2    1    1      0     0     1     1     0     1     1 ...     0    0.0       1
3    0    1      1     1     0     0     1     0     0 ...     0    0.0       1
4    0    0      1     0     1     0     1     0     0 ...     0    0.0       1
5    0    0      1     1     0     0     1     0     1 ...     1    0.0       1
6    0    0      1     0     1     0     0     1     1 ...     0    0.0       1
7    0    0      1     0     0     1     0     0     0 ...     1    0.0       1
8    1    0      1     1     1     1     1     0     1 ...     1    0.0       1
9    0    0      0     1     1     0     1     1     1 ...     1    0.0       1
['BINARY', 10 rows, 10 samples, 93 variables]


## Solve on QPU

In [25]:
k=10
# solve problem
solution1, response1 = solve_qubo(H,spins,"QPU",k)

# display result
print(response1)
# for smple, energy, num_occ in response.data(['sample','energy','num_occurrences']):
#     print(num_occ)

# display sums
print("")
S0 = [int(j[j.find('.')+1:]) for (j, xi) in solution1.sample.items() if xi >  0.5]
S1 = [int(j[j.index('.')+1:]) for (j, xi) in solution1.sample.items() if xi <= 0.5]

print('S:', S)
print('S0:',S0)
print('S1:',S1)

print("")

print("sum(S0) %8d" % sum(S0))
print("sum(S1) %8d" % sum(S1))

Solving QUBO problem (100 vars, 900 nz) on QPU...
Solved in 110.73 seconds
  x0.6 x1.3 x10.10 x11.7 x12.4 x13.3 x14.7 x15.7 ... x99.4  energy num_oc. ...
4    1    0      1     0     0     1     1     0 ...     0     4.0       1 ...
6    1    1      0     0     1     1     1     1 ...     1    36.0       1 ...
0    0    0      1     0     1     1     1     0 ...     0    64.0       1 ...
3    0    0      1     1     0     1     1     0 ...     0   196.0       1 ...
7    1    1      1     0     0     1     1     1 ...     0   196.0       1 ...
8    0    1      1     1     1     0     1     0 ...     1   196.0       1 ...
1    1    1      1     1     1     1     1     0 ...     0   400.0       1 ...
5    1    1      1     0     0     0     1     0 ...     0   400.0       1 ...
9    1    0      1     0     1     1     1     0 ...     0   400.0       1 ...
2    1    1      1     0     1     1     1     1 ...     0 12544.0       1 ...
['BINARY', 10 rows, 10 samples, 93 variables]

S: [ 6  3